<a href="https://colab.research.google.com/github/Noopuragr/Project/blob/master/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import re
import pickle
import datetime

df_date=pd.date_range('2009-01-01','2018-11-28', freq='M')
date_range = []
for x in df_date:
    date_range.append(str(x)[:4]+str(x)[5:7])
 

In [0]:
temp_mean,humidity_mean,dewpoint_mean,barometer_mean,windSpeed_mean,gustSpeed_mean,direction_mean,rainfall_month,rainfall_year,rainfall_minute=[],[],[],[],[],[],[],[],[],[]
date_index = []
temp_max,temp_min,humidity_max,humidity_min,pressure_max,pressure_min,windspeed_max,gustspeed_max,heatindex_max=[],[],[],[],[],[],[],[],[]
clean_data_list=[]
 #'Average temperature (°F)', 'Average humidity (%)',
 #'Average dewpoint (°F)', 'Average barometer (in)',
 #'Average windspeed (mph)', 'Average gustspeed (mph)',
 #'Average direction (°deg)', 'Rainfall for month (in)',
 #'Rainfall for year (in)', 'Maximum rain per minute',
 #'Maximum temperature (°F)', 'Minimum temperature (°F)',
 #'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
 #'Minimum pressure', 'Maximum windspeed (mph)',
 #'Maximum gust speed (mph)', 'Maximum heat index (°F)'
    
def clean_data_extractor(final_list,month,parameter):
    for i in final_list:
        daywise_data = []
        for item in i:
            if item.split(' ')[1] == '29' and parameter[4:6] =='10' and parameter[0:4] == '2018':
                return
            else:
                if item.startswith(month):
                    new_ind = ''
                    if len(item.split(' ')[1]) == 1:
                        new_ind = parameter+'0'+item.split(' ')[1]
                    else:
                        new_ind = parameter+item.split(' ')[1]
                    daywise_data.append(pd.to_datetime(new_ind))
                if item.startswith('Average temperature'):
                    avg_temp = item.split(' ')[2].replace("°F", "")
                    daywise_data.append(float(avg_temp))

                if item.startswith('Average humidity'):
                    avg_humidity = item.split(' ')[2].split('%')[0]
                    if avg_humidity =="":
                        avg_humidity = 0
                    daywise_data.append(float(avg_humidity))

                if item.startswith('Average dewpoint'):
                    avg_dewpoint = item.split(' ')[2].replace("°F", "")
                    if avg_dewpoint =="":
                        avg_dewpoint = 0
                    daywise_data.append(float(avg_dewpoint))

                if item.startswith('Average barometer'):
                    avg_barometer = item.split(' ')[2]
                    daywise_data.append(float(avg_barometer))  

                if item.startswith('Average windspeed'):
                    avg_windSpeed = item.split(' ')[2]
                    daywise_data.append(float(avg_windSpeed))

                if item.startswith('Average gustspeed'):
                    avg_gustSpeed = item.split(' ')[2]
                    daywise_data.append(float(avg_gustSpeed))

                if item.startswith('Average direction'):
                    if item.split(' ')[2].split('°')[0] == "":
                        avg_direction = 0
                    else:
                        avg_direction = item.split(' ')[2].split('°')[0]
                    daywise_data.append(float(avg_direction))

                if item.startswith('Rainfall for month'):
                    rain_forMon = item.split(' ')[3]
                    daywise_data.append(float(rain_forMon))

                if item.startswith('Rainfall for year'):
                    rain_forYear = item.split(' ')[3]
                    daywise_data.append(float(rain_forYear))

                if item.startswith('Maximum rain per minute'):
                    rain_forMin = item.split(' ')[4]
                    daywise_data.append(float(rain_forMin))

                if item.startswith('Maximum temperature'):
                    max_temp = item.split(' ')[2].replace("°F", "")
                    daywise_data.append(float(max_temp))

                if item.startswith('Minimum temperature'):
                    min_temp = item.split(' ')[2].replace("°F", "")
                    daywise_data.append(float(min_temp))

                if item.startswith('Maximum humidity'):
                    max_humidity = item.split(' ')[2].split('%')[0]
                    daywise_data.append(float(max_humidity))

                if item.startswith('Minimum humidity'):
                    min_humidity = item.split(' ')[2].split('%')[0]
                    if min_humidity == "":
                        min_humidity = 0
                    daywise_data.append(float(min_humidity))

                if item.startswith('Maximum pressure'):
                    max_pre = item.split(' ')[2]
                    daywise_data.append(float(max_pre))

                if item.startswith('Minimum pressure'):
                    min_pres = item.split(' ')[2]
                    daywise_data.append(float(min_pres))

                if item.startswith('Maximum windspeed'):
                    max_windSpeed = item.split(' ')[2]
                    daywise_data.append(float(max_windSpeed))

                if item.startswith('Maximum gust speed'):
                    max_gustSpeed = item.split(' ')[3]
                    if max_gustSpeed == "":
                        max_gustSpeed=0
                    daywise_data.append(float(max_gustSpeed))

                if item.startswith('Maximum heat index'):
                    max_heatIndex = item.split(' ')[3].replace("°F", "")
                    daywise_data.append(float(max_heatIndex))
        clean_data_list.append(daywise_data)
    return clean_data_list

In [0]:
for val in date_range:
        url ="http://www.estesparkweather.net/archive_reports.php?date="+val
        resultSet = requests.get(url)
        soup = BeautifulSoup(resultSet.content,"lxml")
        scrapped_data = soup.find_all('table')

        web_data = []
        data_final = []
        month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
        month_num = val[4:6]
        mon_idx = int(month_num)-1
        for i in scrapped_data:
            web_data.append(i.text.splitlines())
        for x in web_data:
            while x.count(''):
                  x.remove('')
        for x in web_data:           
            if x[0].startswith(month[mon_idx]):
                data_final.append(x)
        #print(final_list)
        clean_data_extractor(data_final,month[mon_idx],val)

In [0]:
for x in clean_data_list:
        #print(records)
        date_index.append(x[0])
        temp_mean.append(x[1])
        humidity_mean.append(x[2])
        dewpoint_mean.append(x[3])
        barometer_mean.append(x[4])
        windSpeed_mean.append(x[5])
        gustSpeed_mean.append(x[6])
        direction_mean.append(x[7])
        rainfall_month.append(x[8])
        rainfall_year.append(x[9])
        rainfall_minute.append(x[10])
        temp_max.append(x[11])
        temp_min.append(x[12])
        humidity_max.append(x[13])
        humidity_min.append(x[14])
        pressure_max.append(x[15])
        pressure_min.append(x[16])
        windspeed_max.append(x[17])
        gustspeed_max.append(x[18])
        heatindex_max.append(x[19])

In [0]:
pd_final = pd.DataFrame({'Average temperature (°F)':temp_mean,'Average humidity (%)':humidity_mean,'Average dewpoint (°F)':dewpoint_mean,
                      'Average barometer (in)':barometer_mean,'Average windspeed (mph)':windSpeed_mean,'Average gustspeed (mph)':gustSpeed_mean,
                      'Average direction (°deg)':direction_mean,'Rainfall for month (in)':rainfall_month,'Rainfall for year (in)':rainfall_year,
                      'Maximum rain per minute':rainfall_minute,'Maximum temperature (°F)':temp_max,'Minimum temperature (°F)':temp_min,
                     'Maximum humidity (%)':humidity_max,'Minimum humidity (%)':humidity_min,'Maximum pressure':pressure_max,
                     'Minimum pressure':pressure_min,'Maximum windspeed (mph)':windspeed_max,'Maximum gust speed (mph)':gustspeed_max,
                     'Maximum heat index (°F)':heatindex_max},index=date_index)

In [0]:
with open("dataframe.pk", "wb") as file:
    pickle.dump(pd_final, file)

In [10]:
pd_final.head()

,Average barometer (in),Average dewpoint (°F),Average direction (°deg),Average gustspeed (mph),Average humidity (%),Average temperature (°F),Average windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F),Maximum humidity (%),Maximum pressure,Maximum rain per minute,Maximum temperature (°F),Maximum windspeed (mph),Minimum humidity (%),Minimum pressure,Minimum temperature (°F),Rainfall for month (in),Rainfall for year (in)
2009-01-01,29.7,12.7,274.0,36.8,35.0,37.8,26.4,59.0,40.1,44.0,29.762,0.0,40.1,41.4,27.0,29.596,34.5,0.0,0.0
2009-01-02,29.5,14.7,240.0,18.0,32.0,43.2,12.8,51.0,52.8,43.0,29.669,0.0,52.8,35.7,16.0,29.268,37.5,0.0,0.0
2009-01-03,29.7,12.7,290.0,12.2,60.0,25.7,8.3,38.0,41.2,89.0,30.232,0.0,41.2,25.3,35.0,29.260,6.7,0.0,0.0
2009-01-04,30.4,0.1,47.0,4.5,67.0,9.3,2.9,20.0,32.0,79.0,30.566,0.0,19.4,12.7,35.0,30.227,-0.0,0.0,0.0
2009-01-05,29.9,-5.3,265.0,23.1,30.0,23.5,16.7,53.0,32.0,56.0,30.233,0.0,30.3,38.0,13.0,29.568,15.1,0.0,0.0


In [13]:
#Standard deviation of maximum windspeed
print("Standard deviation of maximum windspeed",pd_final['Maximum windspeed (mph)'].std())

Standard deviation of maximum windspeed 13.059215714829437


In [21]:
#pearson correlation between avg dew point and avg temperature
data_corr=pd_final[["Average dewpoint (°F)","Average temperature (°F)"]]
x=data_corr.corr(method='pearson')
print("pearson correlation between avg dew point and avg temperature",x.values[0][1])

pearson correlation between avg dew point and avg temperature 0.7596091253567293


In [25]:
#difference between 50th percentile and 75th percentile of aveerage temperature
x = pd_final["Average temperature (°F)"].quantile(0.5)
y= pd_final["Average temperature (°F)"].quantile(0.75)
print("difference between 50th percentile and 75th percentile of aveerage temperature",(x-y))


difference between 50th percentile and 75th percentile of aveerage temperature -12.200000000000003


In [45]:
x = pd_final["Average humidity (%)"].argmin().month
print("the lowest average humidity occured in month",x)


the lowest average humidity occured in month 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  """Entry point for launching an IPython kernel.


In [0]:
pd_final["pressure_diff"] = (pd_final["Maximum pressure"]-pd_final["Minimum pressure"])

In [47]:
pd_final.head()

,Average barometer (in),Average dewpoint (°F),Average direction (°deg),Average gustspeed (mph),Average humidity (%),Average temperature (°F),Average windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F),Maximum humidity (%),Maximum pressure,Maximum rain per minute,Maximum temperature (°F),Maximum windspeed (mph),Minimum humidity (%),Minimum pressure,Minimum temperature (°F),Rainfall for month (in),Rainfall for year (in),pressure_diff
2009-01-01,29.7,12.7,274.0,36.8,35.0,37.8,26.4,59.0,40.1,44.0,29.762,0.0,40.1,41.4,27.0,29.596,34.5,0.0,0.0,0.166
2009-01-02,29.5,14.7,240.0,18.0,32.0,43.2,12.8,51.0,52.8,43.0,29.669,0.0,52.8,35.7,16.0,29.268,37.5,0.0,0.0,0.401
2009-01-03,29.7,12.7,290.0,12.2,60.0,25.7,8.3,38.0,41.2,89.0,30.232,0.0,41.2,25.3,35.0,29.260,6.7,0.0,0.0,0.972
2009-01-04,30.4,0.1,47.0,4.5,67.0,9.3,2.9,20.0,32.0,79.0,30.566,0.0,19.4,12.7,35.0,30.227,-0.0,0.0,0.0,0.339
2009-01-05,29.9,-5.3,265.0,23.1,30.0,23.5,16.7,53.0,32.0,56.0,30.233,0.0,30.3,38.0,13.0,29.568,15.1,0.0,0.0,0.665


In [0]:
import datetime as dt
from datetime import date
import calendar

In [60]:
#import calenda
x = pd_final["pressure_diff"].argmax()
#calendar.day_name(x)
print("Highest diff between max pressure and min pressure occured in ",x)

Highest diff between max pressure and min pressure occured in  2018-03-23 00:00:00


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  """Entry point for launching an IPython kernel.


In [0]:
x= pd_final["Maximum gust speed (mph)"].median()


In [79]:
#average temperature between maech 2010 to may 2012

#st = datetime.datetime(2010, 3, 31)
#en = datetime.datetime(2012, 5, 31)

#pd_final.loc[st:en]


start = pd_final.index.searchsorted(dt.datetime(2010, 3, 31))

end = pd_final.index.searchsorted(dt.datetime(2012, 5, 31))

x = pd_final.ix[start:end]

print("Average temperature between march 2010 and may 2012 is",x["Average temperature (°F)"].mean())

Average temperature between march 2010 and may 2012 is 45.87903726708072


In [91]:
#How many days fall under median of barometer reading
x=pd_final["Average barometer (in)"].median()

y = pd_final[pd_final["Average barometer (in)"]==x]
print("No. of days fall under median of barometer reading is ",y["Average barometer (in)"].count())

No. of days fall under median of barometer reading is  3204


In [99]:
#Average temperature on dec 2010

x = pd_final[(pd_final.index.year==2010) & (pd_final.index.month==12)]
print("Average temperature on dec 2010 is", x["Average temperature (°F)"].mean())



Average temperature on dec 2010 is 31.276666666666667


In [108]:
#How many days are within standard deviation of average temperature

x = pd_final["Average temperature (°F)"].std(ddof=1)
#x
pd_temp = pd_final[pd_final["Average temperature (°F)"]==x]
#pd_temp.count()
pd_temp
print("Number of days fall within std deviation of average temperature is", pd_temp["Average temperature (°F)"].count())

Number of days fall within std deviation of average temperature is 0
